<a href="https://colab.research.google.com/github/brunotech/BioBERTpt/blob/master/prophet_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

     |████████████████████████████████| 6.3 MB 8.3 MB/s 


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [3]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-08-26,"$3,228.75","$3,249.66","$3,060.23","$3,100.33","$3,100.33",17405668117
2021-08-27,"$3,096.41","$3,281.84","$3,064.49","$3,270.60","$3,270.60",18489602004
2021-08-28,"$3,275.10","$3,284.85","$3,217.40","$3,244.40","$3,244.40",13709633698
2021-08-29,"$3,246.77","$3,283.24","$3,158.84","$3,227.00","$3,227.00",13296586731
2021-08-31,"$3,219.03","$3,246.24","$3,199.93","$3,222.83","$3,222.83",19828520960


In [4]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2065 entries, 2016-01-01 to 2021-08-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2065 non-null   float64
 1   High       2065 non-null   float64
 2   Low        2065 non-null   float64
 3   Close      2065 non-null   float64
 4   Adj Close  2065 non-null   float64
 5   Volume     2065 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 112.9 KB


In [5]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [6]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [7]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [8]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [9]:
df.tail()

,ds,y
2060,2021-08-26,"$3,228.75"
2061,2021-08-27,"$3,096.41"
2062,2021-08-28,"$3,275.10"
2063,2021-08-29,"$3,246.77"
2064,2021-08-31,"$3,219.03"


In [10]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2425,2022-08-27
2426,2022-08-28
2427,2022-08-29
2428,2022-08-30
2429,2022-08-31


In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2425,2022-08-27,"$5,674.14","$4,724.57","$6,586.35"
2426,2022-08-28,"$5,573.33","$4,664.39","$6,507.59"
2427,2022-08-29,"$5,448.73","$4,569.17","$6,349.06"
2428,2022-08-30,"$5,376.14","$4,497.06","$6,289.22"
2429,2022-08-31,"$5,201.55","$4,323.44","$6,090.04"


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

2756.1219327019594

In [15]:
plot_plotly(m, forecast)

In [16]:
plot_components_plotly(m, forecast)